In [1]:
import torch
from d2l import torch as d2l

In [2]:
devices = d2l.try_all_gpus()

def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

IndexError: list index out of range

In [3]:
run(x_gpu1)
run(x_gpu2) # 预热设备 
torch.cuda.synchronize(devices[0]) 
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

NameError: name 'x_gpu2' is not defined

In [4]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 & GPU2: 4.6740 sec


NameError: name 'x_gpu2' is not defined

In [5]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在GPU1上运行'): 
    y = run(x_gpu1)
    torch.cuda.synchronize()
    
with d2l.Benchmark('复制到CPU'): 
    y_cpu = copy_to_cpu(y) 
    torch.cuda.synchronize()

在GPU1上运行: 4.2842 sec


AssertionError: Torch not compiled with CUDA enabled

In [6]:
with d2l.Benchmark('在GPU1上运行并复制到CPU'): 
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()

在GPU1上运行并复制到CPU: 4.4835 sec


AssertionError: Torch not compiled with CUDA enabled